<a href="https://colab.research.google.com/github/KlaraDewitte/Machine_Learning_course_UGent_D012554_kaggle/blob/master/Poging6SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
trainset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_train.csv")

testset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_test.csv")

In [0]:
labels = trainset.pop('label')
indices = testset.pop('index')

In [0]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
scaler.fit(trainset)
trainset_scaled = pd.DataFrame(scaler.transform(trainset))
testset_scaled = pd.DataFrame(scaler.transform(testset))


In [0]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.model_selection import cross_val_score

In [0]:
X = trainset_scaled
y = labels

In [0]:
params_grid = {'C': [0.01, 0.1, 1, 10, 100, 1000],
          'gamma': [0.01, 0.1, 1, 10],
          'kernel':['linear', 'poly', 'rbf', 'sigmoid'] }


grid_clf = GridSearchCV(SVC(class_weight='balanced'), params_grid)
grid_clf.fit(X, y)

print(grid_clf.best_estimator_)


SVC(C=10, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=False)


In [0]:
clf=SVC(10, gamma=1, kernel='rbf')
clf.fit(X,y)
score_auc = np.mean(cross_val_score(clf,X,y,cv=10,scoring='roc_auc'))
print(score_auc)

0.9624199847043885


In [0]:
clf=SVC(C=100, gamma=0.1, kernel='rbf', probability=True)
clf.fit(X,y)
predictions = clf.predict_proba(testset_scaled)[:,1]
print(predictions)

[2.03041906e-06 8.30271829e-01 5.47867048e-03 ... 9.99988970e-01
 5.10189299e-01 1.99764899e-02]


In [0]:
dfpredictions= pd.DataFrame(({'index': indices, 'label':predictions}))
print(dfpredictions)

       index     label
0          0  0.000002
1          1  0.830272
2          2  0.005479
3          3  1.000000
4          4  1.000000
...      ...       ...
12887  12887  0.072748
12888  12888  0.999998
12889  12889  0.999989
12890  12890  0.510189
12891  12891  0.019976

[12892 rows x 2 columns]


In [0]:
filename = "my_prediction_results_Poging6SVM.csv"

#make sure to not write the Pandas index column (index=False)
dfpredictions.to_csv(filename,index=False)